# Snake Tutorial Solutions

shhh don't tell the kids

In [1]:
# Import statements

from tkinter import Tk, Canvas, Frame, BOTH
from random import randint

In [2]:
class Block:
    def __init__(self, x, y, size=20):
        self.x = x
        self.y = y
        self.size = size
    
    # Already completed
    def draw(self, canvas, color):
        x1 = self.x * self.size
        y1 = self.y * self.size
        x2 = x1 + self.size
        y2 = y1 + self.size
        return canvas.create_rectangle(x1, y1, x2, y2, outline='', fill=color)
    
    def equals(self, other):
        return self.x == other.x and self.y == other.y

In [8]:
class Game:

    def __init__(self, start_speed=10, width=50, height=40, size=20):
        self.speed = start_speed
        self.score = 0
        self.width = width
        self.height = height
        self.snake = Snake(width // 2, height // 2)
        self.food = Food(width, height)
        
        self.create_canvas(width, height, size)
    
    # Already completed
    def create_canvas(self, width, height, size):
        def on_press(event):
            key = event.keysym
            if key in ['Left', 'Right', 'Up', 'Down']:
                self.snake.move(key)
            elif key == 'q':
                window.destroy()
        
        self.window = Tk()
        self.window.geometry('{}x{}'.format(width * size, height * size))
        self.window.resizable(False, False)
        self.window.bind('<Key>', on_press)

        frame = Frame(self.window)
        frame.master.title('Snake')
        frame.pack(fill=BOTH, expand=1)

        self.canvas = Canvas(frame)
        self.canvas.pack(fill=BOTH, expand=1)
        
        
    def start(self):
        self.tick()
        self.window.mainloop()
    
    def crashed(self):
        head = self.snake.head()
        for b in self.snake.blocks[:-1]:
            if head.equals(b):
                return True
        in_bounds = 0 <= head.x < self.width and 0 <= head.y < self.height
        return not in_bounds
    
    def tick(self):
        self.snake.update(self.food)
        if self.crashed():
            print(f'Snake died with score {self.score}')
            self.window.destroy()
        if self.food.eaten(self.snake):
            self.food.__init__(self.width, self.height)
            self.score += 1
            self.speed += 1
        self.render()
        self.window.after(int(1000 / self.speed), self.tick)
        
    def render(self):
        self.canvas.delete('all')
        self.snake.draw(self.canvas)
        self.food.draw(self.canvas)

In [9]:
class Food:
    def __init__(self, width, height, size=20):
        self.block = Block(randint(0, width - 1), randint(0, height - 1), size)
    
    def draw(self, canvas):
        self.block.draw(canvas, 'red')
        
    def eaten(self, snake):
        return self.block.equals(snake.head())

In [10]:
class Snake:
    def vertical(direction):
        return direction == 'Up' or direction == 'Down'
    
    def __init__(self, startX, startY):
        self.blocks = [Block(startX + i, startY) for i in range(4)]
        self.direction = None
    
    def move(self, direction):
        if Snake.vertical(direction) != Snake.vertical(self.direction):
            self.direction = direction
    
    def head(self):
        return self.blocks[-1]

    def update(self, food):
        x = self.head().x
        y = self.head().y
        
        if self.direction == 'Left':
            next_block = Block(x - 1, y)
        elif self.direction == 'Right':
            next_block = Block(x + 1, y)
        elif self.direction == 'Up':
            next_block = Block(x, y - 1)
        elif self.direction == 'Down':
            next_block = Block(x, y + 1)
        else:
            return
        
        self.blocks.append(next_block)

        if next_block.equals(food.block):
            pass
        else:
            self.blocks.pop(0)
    
    def draw(self, canvas):
        for block in self.blocks:
            block.draw(canvas, 'grey')

In [13]:
game = Game()
game.start()

Snake died with score 1


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "<ipython-input-8-b3e8f9428e87>", line 56, in tick
    self.render()
  File "<ipython-input-8-b3e8f9428e87>", line 60, in render
    self.canvas.delete('all')
  File "/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 2514, in delete
    self.tk.call((self._w, 'delete') + args)
_tkinter.TclError: invalid command name ".!frame.!canvas"
